
# H1 Stable Rank (via Subsampling) + SVM on STFT Point Clouds

This notebook mirrors the style of *lecture_15* workflows, but applies them to your **STFT** data:
1. Treat each spectrogram (**1000×257**) as a **point cloud** of 1000 points in 257D (each time frame is a point).
2. Compute **H1 stable rank** using **subsampling** + **Vietoris–Rips persistence** (via `ripser`).
3. Train an **SVM** on the resulting **H1 stable rank features** (one feature vector per clip).


In [36]:

# If imports fail, run the install cell below.
import os, json, glob, math, warnings
from pathlib import Path
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed

# TDA / ML
from ripser import ripser                # pip install ripser
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score

warnings.filterwarnings('ignore')
print("Imports OK")

Imports OK


In [37]:

# Install helpers if needed (uncomment the relevant lines in your environment)
# %pip install ripser scikit-learn tqdm joblib
# Conda (alternative):
# !conda install -y -c conda-forge ripser scikit-learn tqdm joblib



## Configuration
Set where your `stft_np/` lives (created by your earlier STFT script).

In [38]:

# === Paths / config ===
BASE_DIR      = '.'
META_PATH     = os.path.join(BASE_DIR, 'stft_np', 'meta.json')
A_MEMMAP_PATH = None
B_MEMMAP_PATH = None

# Subsample strategy for Stable Rank
SUBSAMPLE_SIZE    = 150     # m points per subsample
N_SUBSAMPLES      = 6       # subsamples per clip
T_POINTS          = 64      # number of t-grid points for the stable-rank curve
RANDOM_STATE      = 42

# Dataset usage (adjust for speed)
N_PER_CLASS       = 1000     # clips per class to use (raise when ready)
TEST_SIZE         = 0.2

# Pre-scan parameters to set global t-grid range
PRESCAN_SAMPLES   = 100     # number of clips to probe for t_max estimate
PRESCAN_SUBS      = 2
PRESCAN_M         = 100

# Homology coefficient field (prime only): 2, 3, 5, 7, ...
COEFF = 2

np.set_printoptions(precision=4, suppress=True)
print("Configured.")

Configured.



## Load STFT arrays and pick subsets
Each clip is `(1000, 257)` in **dB** ([-80, 0]). We will convert each to a point cloud in `[0,1]` or z-scored space.

In [39]:

# === Load meta & memmaps ===
with open(META_PATH, 'r') as f:
    meta = json.load(f)

dtype_str = meta.get('dtype', 'float16')
if isinstance(dtype_str, str) and dtype_str.startswith("<class 'numpy."):
    dtype_str = dtype_str.split('.')[-1].split("'")[0]

A_MEMMAP_PATH = os.path.join(BASE_DIR, 'stft_np', meta['A_memmap'])
B_MEMMAP_PATH = os.path.join(BASE_DIR, 'stft_np', meta['B_memmap'])
A_shape = tuple(meta['A_shape'])
B_shape = tuple(meta['B_shape'])

A_mm = np.memmap(A_MEMMAP_PATH, dtype=dtype_str, mode='r', shape=A_shape)
B_mm = np.memmap(B_MEMMAP_PATH, dtype=dtype_str, mode='r', shape=B_shape)

print("A:", A_mm.shape, "B:", B_mm.shape)

A: (940, 1000, 257) B: (973, 1000, 257)


In [40]:

# === Choose subset indices and labels ===
rng = np.random.default_rng(RANDOM_STATE)
nA, nB = A_mm.shape[0], B_mm.shape[0]
nA_use = min(N_PER_CLASS, nA)
nB_use = min(N_PER_CLASS, nB)

idxA = rng.choice(nA, size=nA_use, replace=False)
idxB = rng.choice(nB, size=nB_use, replace=False)

labels = np.concatenate([np.zeros(nA_use, dtype=int), np.ones(nB_use, dtype=int)])
print(f"Using {nA_use} from A and {nB_use} from B → total {labels.size}")

Using 940 from A and 973 from B → total 1913



## Point cloud conversion
We treat each **time frame** (length-257 vector) as one point. We map dB to [0,1] and then **z-score per clip** (across the 1000 frames) so distances are meaningful. Finally, we optionally **L2-normalize** per point.

In [41]:

def spectrogram_to_point_cloud(db_img, zscore=True, l2norm=True):
    """db_img: (1000, 257) in dB [-80,0].
    Returns a (1000, 257) float32 point cloud.
    """
    # Map to [0,1]
    x = (db_img + 80.0) / 80.0
    x = np.clip(x, 0.0, 1.0).astype(np.float32, copy=False)  # (T=1000, F=257)

    if zscore:
        mu = x.mean(axis=0, keepdims=True)
        sd = x.std(axis=0, keepdims=True) + 1e-6
        x = (x - mu) / sd

    if l2norm:
        nrm = np.linalg.norm(x, axis=1, keepdims=True) + 1e-9
        x = x / nrm

    return x.astype(np.float32, copy=False)

# quick smoke test on one clip
_ = spectrogram_to_point_cloud(np.array(A_mm[0], dtype=np.float32))
print("Point-cloud converter ready.")

Point-cloud converter ready.



## Vietoris–Rips H1 diagrams and **Stable Rank** via subsampling
**Stable rank curve** here means: for a grid of radii `t`, average across subsamples the **Betti-1** rank (number of H1 intervals alive at `t`).

In [42]:

def h1_diagram(points, maxdim=1):
    """Compute H1 diagram for a point cloud using ripser.
    Returns an (n_pairs, 2) array [birth, death].
    """
    # ripser expects (n_points, n_dims), metric='euclidean' by default
    R = ripser(points, maxdim=maxdim, coeff=COEFF)
    dgms = R['dgms']
    H1 = dgms[1] if len(dgms) > 1 else np.empty((0,2), dtype=np.float64)
    # ensure finite
    if H1.size:
        # Replace inf deaths with large sentinel (max finite death * 1.05)
        finite = np.isfinite(H1[:,1])
        if finite.any():
            m = H1[finite,1].max()
            H1[~finite,1] = m * 1.05
        else:
            H1 = np.empty((0,2), dtype=np.float64)
    return H1

def betti_curve(diag, t_vals):
    """Given H1 diagram (birth, death), compute Betti-1(t) for each t.
    Returns an array of shape (len(t_vals),).
    """
    if diag.size == 0:
        return np.zeros_like(t_vals, dtype=np.float32)
    b = diag[:,0][:,None]  # (k,1)
    d = diag[:,1][:,None]  # (k,1)
    t = t_vals[None,:]     # (1,T)
    alive = (b <= t) & (t < d)  # (k,T)
    return alive.sum(axis=0).astype(np.float32)

def stable_rank_via_subsampling(pc, t_vals, m=150, n_subs=6, rng=None):
    N, D = pc.shape
    rng = np.random.default_rng() if rng is None else rng
    # ensure rows >= cols to avoid Ripser warning
    m_eff = min(N, max(m, D + 1))

    acc = np.zeros_like(t_vals, dtype=np.float32)
    for _ in range(n_subs):
        idx = rng.choice(N, size=m_eff, replace=False)
        diag = h1_diagram(pc[idx])
        acc += betti_curve(diag, t_vals)
    return (acc / n_subs).astype(np.float32)
    
print("Stable-rank helpers ready.")

Stable-rank helpers ready.



## Set a **global** radius grid `t`

We estimate a **common** `t_max` by a small pre-scan across a few clips, then define `t_vals = linspace(0, t_max, T_POINTS)`. This aligns features across all samples.


In [43]:

def estimate_tmax(mmA, mmB, idxA, idxB, prescan_samples=100, subs=2, m=100, seed=RANDOM_STATE):
    rng = np.random.default_rng(seed)
    # pick subset of clips to scan
    idsA = rng.choice(len(idxA), size=min(prescan_samples//2, len(idxA)), replace=False)
    idsB = rng.choice(len(idxB), size=min(prescan_samples - len(idsA), len(idxB)), replace=False)
    deaths = []
    for i in idsA:
        pc = spectrogram_to_point_cloud(np.array(mmA[idxA[i]], dtype=np.float32))
        for _ in range(subs):
            I = rng.choice(pc.shape[0], size=min(m, pc.shape[0]), replace=False)
            H1 = h1_diagram(pc[I])
            if H1.size:
                deaths.append(np.max(H1[:,1]))
    for i in idsB:
        pc = spectrogram_to_point_cloud(np.array(mmB[idxB[i]], dtype=np.float32))
        for _ in range(subs):
            I = rng.choice(pc.shape[0], size=min(m, pc.shape[0]), replace=False)
            H1 = h1_diagram(pc[I])
            if H1.size:
                deaths.append(np.max(H1[:,1]))
    if len(deaths) == 0:
        return 1.0
    deaths = np.array(deaths, dtype=np.float64)
    tmax = float(np.quantile(deaths, 0.95))
    return max(tmax, 1e-3)

t_max = 2.0 #estimate_tmax(A_mm, B_mm, idxA, idxB, prescan_samples=PRESCAN_SAMPLES, subs=PRESCAN_SUBS, m=PRESCAN_M)
t_vals = np.linspace(0.0, t_max, T_POINTS).astype(np.float32)
print("t_max≈", t_max, "grid length:", len(t_vals))

t_max≈ 2.0 grid length: 64



## Compute H1 stable rank features
One **(T_POINTS, )** feature vector **per clip**.


In [44]:

def features_for_indices(mm, indices, t_vals, m=SUBSAMPLE_SIZE, n_subs=N_SUBSAMPLES, n_jobs=-1, desc='features'):
    def one(idx):
        db = np.array(mm[idx], dtype=np.float32)
        pc = spectrogram_to_point_cloud(db)
        return stable_rank_via_subsampling(pc, t_vals, m=m, n_subs=n_subs, rng=np.random.default_rng(RANDOM_STATE+idx))
    out = Parallel(n_jobs=n_jobs, prefer='processes')(
        delayed(one)(i) for i in tqdm(indices, desc=desc)
    )
    return np.vstack(out).astype(np.float32, copy=False)

print("Computing A features...")
XA = features_for_indices(A_mm, idxA, t_vals, desc='A stable-rank')
print("Computing B features...")
XB = features_for_indices(B_mm, idxB, t_vals, desc='B stable-rank')

X = np.vstack([XA, XB]).astype(np.float32, copy=False)
y = labels.copy()
print("X:", X.shape, "y:", y.shape)

Computing A features...


A stable-rank: 100%|██████████| 940/940 [00:37<00:00, 24.96it/s]


Computing B features...


B stable-rank: 100%|██████████| 973/973 [00:31<00:00, 31.33it/s]


X: (1913, 64) y: (1913,)



## Train / Test split and SVM fit

In [47]:

sss = StratifiedShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=RANDOM_STATE)
train_idx, test_idx = next(sss.split(X, y))

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

clf = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=10.0, gamma='scale', probability=False, random_state=RANDOM_STATE)
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 (weighted):", f1_score(y_test, y_pred, average='weighted'))
print()
print(classification_report(y_test, y_pred, target_names=['A','B']))

Accuracy: 0.6762402088772846
F1 (weighted): 0.6762666959068283

              precision    recall  f1-score   support

           A       0.67      0.68      0.67       188
           B       0.69      0.67      0.68       195

    accuracy                           0.68       383
   macro avg       0.68      0.68      0.68       383
weighted avg       0.68      0.68      0.68       383




## Save features (optional)

In [46]:

os.makedirs('stft_np/tda_features', exist_ok=True)
np.save('stft_np/tda_features/X_stable_rank.npy', X)
np.save('stft_np/tda_features/y_labels.npy', y)
np.save('stft_np/tda_features/t_grid.npy', t_vals)
print("Saved to stft_np/tda_features/")

Saved to stft_np/tda_features/
